<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# Introduction

For this assignment, as you did in the first lab, **do not use**:



*   the native fuctions <code>map()</code> and <code>reduce()</code> of python
*   the Pandas dataframes
*   the PySpark dataframes.








# Setting the environment

[Kaggle](https://www.kaggle.com/) is a data science competition platform and online community of data scientists and machine learning practitioners under Google LLC. In Kaggle you can also find and publish datasets. For our exercises we will often use datasets published on Kaggle.

## A. Environment set-up

It is possible to folllow the following procedure that includes directly Kaggle data into colab working folders.

### Step 1: Download the token from Kaggle

To download data from Kaggle directly in your Colab environment you need to follow these steps:

1.   Authenticate with the Kaggle services (if you do not have an account you must create one)
2.   Get your API token.

The token can be easily generated and downloaded from the profile section of your Kaggle account.

Simply, navigate to your Kaggle profile and then, **click** the **Account** tab and then scroll down to the API section.

A file named <code>kaggle.json</code> will be download.

This file contains the username and the API key.

This is a one-time step and you don’t need to generate the credentials every time you download the dataset.


Upload the file <code>kaggle.json</code> that you just downloaded from Kaggle in the data folder of your Colab environment and execute the following commands.

In [1]:
! pip install kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

mkdir: /Users/zhengwan/.kaggle: File exists


## B. Importing data

Execute the following line to import data, we will discuss the database format in the following paragraphs.



In [2]:
! kaggle datasets download uciml/sms-spam-collection-dataset

100%|████████████████████████████████████████| 211k/211k [00:00<00:00, 1.37MB/s]
100%|████████████████████████████████████████| 211k/211k [00:00<00:00, 1.36MB/s]


In [3]:
! unzip sms-spam-collection-dataset.zip

Archive:  sms-spam-collection-dataset.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# Exercise: Messages and statistics

<p align="justify">
<font size="3">
Suppose now that you have a text file generated by a monitoring system of a company.

Each line represents a tagged message that have been collected from SMS Spam research.

Each message can me tagged as ham (legitimate) or spam.

The items are separated by a comma.

A sample line:

`ham,There are many company. Tell me the language.,,,`

</font>
</p>





## Part 1 - Count Spam - Ham
<p align="justify">
<font size="3">
Describe and implement a Map-Reduce algorithm that provides the total number of spam messages and ham messages:

```
Ham: ...
Spam: ...
```


</font>
</p>


<p>

**Hint**:

Read the .csv file line by line and think about how to consider the keys.
</p>

In [7]:
# Write here any support function you need to read the .csv file and/or format your text
import csv

def read_csv_file(file_path):
    """
    Read CSV file and return data

    Parameters
    ----------
    file_path

    Return
    ----------
    list
    """
    data = []
    with open(file_path, 'r', encoding='ISO-8859-1') as csv_file:
      csv_reader = csv.reader(csv_file)
      for row in csv_reader:
        data.append(row)
    return data



data = read_csv_file('spam.csv')

print (data[:5])

[['v1', 'v2', '', '', ''], ['ham', 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', '', '', ''], ['ham', 'Ok lar... Joking wif u oni...', '', '', ''], ['spam', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", '', '', ''], ['ham', 'U dun say so early hor... U c already then say...', '', '', '']]


### <strong> 1.1. Map </strong>

In [9]:
#Provide here few comment lines that describe your map function

def  map_num_spam_ham(data):
  '''
  This function takes a dataset as input and maps the first element of each data row (typically "ham" or "spam") to a key-value pair,
  where the key is that element, and the value is 1.

  param
  -----
  data: Input dataset, typically a list or RDD containing multiple rows of data.

  return: Returns a list containing key-value pairs, where the key is the first element of the data, and the value is 1.
  '''

  res =[]
  for i in data[1:]:
    value = i[0]
    res.append((value,1))
  return res


res1 = map_num_spam_ham(data)
res1[:15]

[('ham', 1),
 ('ham', 1),
 ('spam', 1),
 ('ham', 1),
 ('ham', 1),
 ('spam', 1),
 ('ham', 1),
 ('ham', 1),
 ('spam', 1),
 ('spam', 1),
 ('ham', 1),
 ('spam', 1),
 ('spam', 1),
 ('ham', 1),
 ('ham', 1)]

### <strong> 1.2. Shuffle </strong>

In [ ]:
#Provide here few comment lines that describe your shuffle function
from collections import defaultdict

def  shuffle_num_spam_ham(input):
  '''
  This function takes a list of key-value pairs as input and performs a shuffle operation to group the values by their corresponding keys.

  param
  -----
  input: A list of key-value pairs to be shuffled.
  return: Returns a dictionary where keys are unique keys from the input, and values are lists of associated values.
  '''
  dict= defaultdict(list)
  for key,value in input:
    dict[key].append(value)
  return dict




res2 = shuffle_num_spam_ham(res1)
res2


### <strong> 1.3. Reduce </strong>

In [14]:
#Provide here few comment lines that describe your reduce function

def  reduce_num_spam_ham(input):
    '''
    This function takes a dictionary and reduces the data by counting the number of elements in each list and returns a list of key-value pairs
    :param input: A dictionary where keys represent categories or values, and values are lists of associated data.
    :return: Returns a list of key-value pairs, where each key is a category or value, and the value is the count of associated elements.

    '''
    res = []
    for key in input.keys():
      res.append((key,len(input[key])))
    return res


res3 = reduce_num_spam_ham(res2)
res3

[('ham', 4825), ('spam', 747)]


## Part 2 - Number of word in ham and spam messages

<p align="justify">
<font size="3">
Describe and implement an algorithm that, using the Map-Reduce paradigm, lists the total number of word in spam messages and the total number of word in ham messages.

You can decide if removing stop words is significant or not for this analysis.
</font>
</p>



### <strong> 2.1. Map </strong>

In [18]:
#Provide here few comment lines that describe your map function
import string

def  map_words_in_msg(input):
  """
Maps input data to lowercase, removes punctuation, and splits text into words.

Args:
    input (list of tuples): Input data where each tuple contains (label, text).

Returns:
    list of tuples: Each tuple contains a label and a list of words.
"""
  res_1, res_2 =[], []
  for i in input:
    if i[0] == 'spam' :
      # Convert the message to lowercase, remove punctuation, and split it into words
      res_1.append(('spam',(i[1].lower().translate(str.maketrans('', '', string.punctuation))).split()))
    elif i[0] == 'ham' :
      res_2.append(('ham',(i[1].lower().translate(str.maketrans('', '', string.punctuation))).split()))
  res_combined = res_1 + res_2
  return res_combined

res21 = map_words_in_msg(data)
res21[:2]


[('spam',
  ['free',
   'entry',
   'in',
   '2',
   'a',
   'wkly',
   'comp',
   'to',
   'win',
   'fa',
   'cup',
   'final',
   'tkts',
   '21st',
   'may',
   '2005',
   'text',
   'fa',
   'to',
   '87121',
   'to',
   'receive',
   'entry',
   'questionstd',
   'txt',
   'ratetcs',
   'apply',
   '08452810075over18s']),
 ('spam',
  ['freemsg',
   'hey',
   'there',
   'darling',
   'its',
   'been',
   '3',
   'weeks',
   'now',
   'and',
   'no',
   'word',
   'back',
   'id',
   'like',
   'some',
   'fun',
   'you',
   'up',
   'for',
   'it',
   'still',
   'tb',
   'ok',
   'xxx',
   'std',
   'chgs',
   'to',
   'send',
   'å£150',
   'to',
   'rcv'])]

### <strong> 2.2. Shuffle </strong>

In [20]:
#Provide here few comment lines that describe your shuffle function

def shuffle_words_in_msg(input):
    """
    This function takes an input dataset consisting of labels and words. It creates a reversed dictionary
    where words are the keys, and the values are lists of (label, word count) pairs.

    :param input: Input data consisting of tuples (label, words)
    :return: A dictionary where each word is a key, and the values are lists of (label, word count) pairs
    """
    reversed_dict = defaultdict(list)
    for label, words in input:
        for word in words:
            word_count = (word, words.count(word))
            reversed_dict[word].append((label, word_count))
    return dict(reversed_dict)

res22 = shuffle_words_in_msg(res21)




free : [('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 3)), ('spam', ('free', 3)), ('spam', ('free', 3)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 1)), ('spam', ('free', 2)), ('spam', ('free', 2)), ('spam', ('free', 1)), ('s

### <strong> 2.3. Reduce </strong>

In [26]:
#Provide here few comment lines that describe your reduce function
def reduce_words_in_msg(dict):
    """
    This function takes a dictionary of word occurrences and categorizes them into 'spam' and 'ham' counts for each word.
    It then sorts the words based on 'ham' occurrences in descending order.

    :param word_counts: Dictionary where keys are words, and values are lists of (label, word count) pairs
    :return: A list of tuples where each tuple contains (word, spam count, ham count) sorted by ham count in descending order
    """
    res = []
    for k in dict.keys():
        counts = dict[k]
        spam_count = len([item for item in counts if item[0] == 'spam'])
        ham_count = len([item for item in counts if item[0] == 'ham'])
        res.append((k, spam_count, ham_count))
    res = sorted(res, key=lambda x: x[2], reverse=True)
    #Sort the word occurrences in 'ham' in descending order
    return res

res32 = reduce_words_in_msg(res22)
print('The ouput interprets like this: (word, spam count, ham count)')
res32[:4]

The ouput interprets like this: (word, spam count, ham count)


[('i', 45, 2185), ('you', 287, 1837), ('to', 686, 1554), ('the', 204, 1118)]


## Part 3 - Average

<p align="justify">
<font size="3">
Describe and implement a Map-Reduce algorithm that gives, the average number of word in spam messages and the everage number of words in ham messages.

You can decide if removing stop words is significant or not for this analysis.

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before.

</font>
</p>



### <strong> 3.1. Map </strong>

In [28]:


stop_words = ["the", "and", "is", "in", "it", "an", "this", "to", "of"]

def map_average(input):
    """
    Maps input data to lowercase, removes punctuation, and splits text into words.

    Args.
        input (list of tuples): Input data containing labels and text.

    Returns.
        list of tuples: Each tuple contains a list of labels and words.
    """
    res_1, res_2 = [], []
    for i in input:
        if i[0] == 'spam':
            # Process to get a list of lowercase words without punctuation and stop words.
            words = [word for word in i[1].lower().translate(str.maketrans('', '', string.punctuation)).split() if word not in stop_words]
            res_1.append(('spam', words))
        elif i[0] == 'ham':
            words = [word for word in i[1].lower().translate(str.maketrans('', '', string.punctuation)).split() if word not in stop_words]
            res_2.append(('ham', words))
    res_combined = res_1 + res_2
    return res_combined

res31 = map_average(data)
res31[:2]


[('spam',
  ['free',
   'entry',
   '2',
   'a',
   'wkly',
   'comp',
   'win',
   'fa',
   'cup',
   'final',
   'tkts',
   '21st',
   'may',
   '2005',
   'text',
   'fa',
   '87121',
   'receive',
   'entry',
   'questionstd',
   'txt',
   'ratetcs',
   'apply',
   '08452810075over18s']),
 ('spam',
  ['freemsg',
   'hey',
   'there',
   'darling',
   'its',
   'been',
   '3',
   'weeks',
   'now',
   'no',
   'word',
   'back',
   'id',
   'like',
   'some',
   'fun',
   'you',
   'up',
   'for',
   'still',
   'tb',
   'ok',
   'xxx',
   'std',
   'chgs',
   'send',
   'å£150',
   'rcv'])]

### <strong> 3.2. Shuffle </strong>

In [30]:
#Provide here few comment lines that describe your shuffle function

def shuffle_average(input):
    """
    Computes the average number of words in each category

    Args:
        input (list of tuples): Input data containing labels and text data.

    Returns:
        list of tuples: Each tuple contains a category label ('spam' or 'ham') and the average number of words.
    """
    res = []
    for i in input:
      res.append((i[0],len(i[1])))
    return res

res32 = shuffle_average(res31)

res32[:5]




[('spam', 24), ('spam', 28), ('spam', 24), ('spam', 25), ('spam', 22)]

### <strong> 3.3. Reduce </strong>

In [31]:
#Provide here few comment lines that describe your reduce function

def  reduce_average(input):
    """
    Computes the average word count for each category (e.g., 'spam' or 'ham') based on the input data.

    Args:
        input (list of tuples): Input data containing labels and word counts.

    Returns:
        list of tuples: Each tuple contains a category label ('spam' or 'ham') and the average word count (rounded to 3 decimal places).
    """
    word_counts = {}
    for label, count in input:
        if label not in word_counts:
            word_counts[label] = {'total_words': 0, 'count': 0}
        word_counts[label]['total_words'] += count
        word_counts[label]['count'] += 1

    averages = []
    for label, counts in word_counts.items():
        average = counts['total_words'] / counts['count']
        averages.append((label, round(average,3)))

    return averages

res33 = reduce_average(res32)
res33

[('spam', 21.578), ('ham', 12.644)]

Look at your code. Can you use a combine operation?

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

### <strong> 3.4. Advanced Combine </strong>

In [45]:
# distribute the data

def distribute_dataset(data, n):
  '''
  split the data into n chunks

  args:
  data->list:data to be distributed
  n->int:numbers of chunks divided

  returns:
  lists of list
  '''
  chunk_size = len(data)//n
  chunks = [data[i:i+chunk_size] for i in range(0,len(data),chunk_size)]
  return chunks

#split the data into 4 chunks
chunks = distribute_dataset(data, 4)


def map_advanced_average(input):
    """
    Maps input data to lowercase, removes punctuation, and splits text into words.
    Removes stop words from the list of words.

    Args:
        input (list of tuples): Input data containing labels and text.

    Returns:
        list of tuples: Each tuple contains a label ('spam' or 'ham') and the count of non-stop words.
    """
    res_1, res_2 = [], []
    for i in input:
        if i[0] == 'spam':

            words = [word for word in i[1].lower().translate(str.maketrans('', '', string.punctuation)).split() if word not in stop_words]
            res_1.append(('spam', len(words)))
        elif i[0] == 'ham':

            words = [word for word in i[1].lower().translate(str.maketrans('', '', string.punctuation)).split() if word not in stop_words]
            res_2.append(('ham', len(words)))
    res_combined = res_1 + res_2
    return res_combined

map_results = [map_advanced_average(chunk) for chunk in chunks]

def shuffle_advanced_average(input):
    """
    Shuffles and combines the intermediate data to accumulate the sum and count of word counts for 'spam' and 'ham' messages.

    Args:
        input (list of tuples): Intermediate data after mapping.

    Returns:
        list of tuples: Each tuple contains a label ('spam' or 'ham'), the sum of word counts, and the count of messages for that label.
    """
    res = []
    sum_spam, sum_ham = 0, 0
    counts_spam, counts_ham =0, 0

    for label, word_count in input:
        if label == 'spam':
            sum_spam += word_count
            counts_spam += 1
        elif label == 'ham':
            sum_ham += word_count
            counts_ham += 1

    res.append(('spam', sum_spam, counts_spam))
    res.append(('ham', sum_ham,counts_ham))

    return res

combined_map_results = []
for result in map_results:
    combined_map_results.extend(result)

def reduce_advanced_average(input):
    """
    Reduces the shuffled data to calculate the average word count for 'spam' and 'ham' messages.
    Rounds the averages to three decimal places.

    Args:
        input (list of tuples): Shuffled and combined data.

    Returns:
        list of tuples: Each tuple contains a label ('spam' or 'ham') and the calculated average word count, rounded to three decimal places.
    """
    res = []
    average_ham, average_spam = 0, 0
    for i in input:
        if i[0] == 'spam':
            average_spam = i[1] / i[2]
        if i[0] == 'ham':
            average_ham = i[1] / i[2]
    res.append(('spam', round((average_spam),3)))
    res.append(('ham', round((average_ham), 3)))
    return res

shuffled_results = shuffle_advanced_average(combined_map_results)
final_results = reduce_advanced_average(shuffled_results)
print(final_results)


[('spam', 21.578), ('ham', 12.644)]
